In [ ]:
!pip install qubovert > h
!pip install pennylane > h

**Exercise 1**.

We want to organise an event where we would like 6 prestigious companies to come: *e1*, ..., *e6*. However, the industry is very competitive and there is some friction between these companies:

- If we invite company *e3*, *e4* will not come.
- If we invite company *e1*, then *e2* and *e3* will not come.
- If we invite company *e2* then neither *e1* nor *e6* will come.


The objective is to decide which companies we invite to maximise the number of companies going to the event.
Model this problem to solve it on a quantum computer. Use COBYLA to optimise the solution.


**Solution 1**


Let $x_i = 1$ if we invite the i-th company and $x_i = 0$ otherwise.
We are looking to maximise:

$$\sum_i x_i$$.

But we have to subtract the attendance of the companies based on the relationships between them:

If $x3 = 1$ and $x4 = 1$ we have to subtract 1 from the attendance and do nothing otherwise. This can be modeled as $-x3 x4$.

In [ ]:
import qubovert

qubo = qubovert.QUBO()

for i in range(1,7):
  qubo.create_var(f"e{i}")

for i in range(1, 7):
  qubo[(f"e{i}",)] = 1

qubo[(f"e{3}", f"e{4}")] = -1

qubo[(f"e{1}", f"e{2}")] = -2
qubo[(f"e{1}", f"e{3}")] = -1

qubo[(f"e{2}", f"e{6}")] = -1

ising = qubo.to_quso()
print("ising", ising)

ising {(): 1.75, (3,): -0.25, (4,): -0.5, (5,): -0.25, (2, 3): -0.25, (0, 1): -0.5, (0, 2): -0.25, (0,): 0.25, (1, 5): -0.25, (1,): 0.25}


In [ ]:
import pennylane as qml

def create_hamiltonian(ising):
  coeffs = []
  ops = []

  for term in ising:
    if len(term) == 0:
      ops.append(qml.Identity(0))
    elif len(term) == 1:
      ops.append(qml.PauliZ(term[0]))
    else:
      ops.append(qml.PauliZ(term[0]) @ qml.PauliZ(term[1]))
    coeffs.append(ising[term])

  return qml.Hamiltonian(coeffs, ops)

H = create_hamiltonian(ising)

In [ ]:
import numpy as np

dev = qml.device("default.qubit", wires = 6)

@qml.qnode(dev)
def circuit(params):
  for i in range(6):
    qml.RY(params[i], wires = i)
  for i in range(5):
    qml.CNOT(wires = [i, i+1])
  return qml.expval(H)

params = np.random.rand(6)
circuit(params)



array(62.78701837)

In [ ]:
from scipy.optimize import minimize

def cost(params):
  return -circuit(params)

sol = minimize(cost, params, method = "COBYLA")
optimal_params = sol.x


In [ ]:
dev = qml.device("default.qubit", wires = 6, shots = 1)

@qml.qnode(dev)
def circuit_sample(params):
  for i in range(6):
    qml.RY(params[i], wires = i)
  for i in range(5):
    qml.CNOT(wires = [i, i+1])
  return qml.sample(wires = range(6))

print("assistants: ", np.round(circuit(optimal_params)))
print("invitations:", circuit_sample(optimal_params))

assistants:  400.0
invitations: [0 0 1 0 1 1]


**Exercise 2**.

Continuing with the previous problem, we know that if we invite the first company, $100$ attendees will come to meet them, if the second company comes, $150$, and the others $120$, $80$, $80$ and $200$ respectively. Your goal will be to replicate the previous exercise but now maximising the number of attendees instead of companies.

In [ ]:
# We are looking to maximise:
# $$\sum_i a_i x_i$$
# where $a_i$ is the number of attendees for company $i$.

qubo = qubovert.QUBO()

attendees = [100, 150, 120, 80, 80, 200]

for i in range(6):
  qubo.create_var(f"e{i+1}")

for i in range(6):
  qubo[(f"e{i+1}",)] = attendees[i]

qubo[(f"e3", f"e4")] = -attendees[2] - attendees[3]  # Correct penalty

qubo[(f"e1", f"e2")] = -attendees[0] - attendees[1]
qubo[(f"e1", f"e3")] = -attendees[0] - attendees[2]

qubo[(f"e2", f"e6")] = -attendees[1] - attendees[5]


ising = qubo.to_quso()
print("ising", ising)

# ... (rest of the code remains the same)


ising {(0,): 67.5, (): 110.0, (1,): 75.0, (2,): 45.0, (3,): 10.0, (4,): -40.0, (5,): -12.5, (2, 3): -50.0, (0, 1): -62.5, (0, 2): -55.0, (1, 5): -87.5}
